<a href="https://colab.research.google.com/github/sohampalande/Deep_Learning-Keras/blob/master/VGGA16_Conv_Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import** **Packages**

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import Adam
#Constants
batch_size=128
num_classes=10
epochs=12
img_rows,img_cols=28,28
channels=1


**Preprocess the Data**

In [0]:
#Load In Data
(x_train,y_train),(x_test,y_test)=mnist.load_data()

#Normalize and Format Data
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels).astype("float32") / 255
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels).astype("float32") / 255
image_shape=(img_rows, img_cols, channels)

#One hot Encoding
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)



**Build Model**

In [0]:
#Build model
model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=image_shape))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) #takes output from previous layer and flatten it into a one dimensional tensor
model.add(Dense(128,activation ='relu'))
model.add(Dense(num_classes,activation ='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])





**Train Model**

In [0]:
model.summary()
model.fit(x_train,y_train,
         batch_size= batch_size,
         epochs=epochs,
         verbose=2,
         validation_data=(x_test,y_test))
score=model.evaluate(x_test,y_test)
print(score)
#This model wors fine but for more complex images we would require a much more complex Convolutional Neural Network
#We will use the VGG16 classifier and tune it to predict images into two categories

**Building and Fine Tuning VGG16 Image Classifier**

In [0]:
#VGG16 won the 2016 Image NET Competition for image classification on 1000 categories
from keras.applications.vgg16 import VGG16
vgg16_model=VGG16()
#keras has a library of fine tuned models which we can import for purposes of fine tuning 
#weights of the model are already defined


In [0]:
vgg16_model.summary()

In [5]:
#model consists of multiple layers
#note the last layer has 1000 output/classification categories
#we want to tune the model to classify only on 2 categories
type(vgg16_model)

keras.engine.training.Model

In [0]:
#this is a model of type "Model" as opposed to being "Sequential" etc...
#we will transform the model to a sequential model

new_model=Sequential()   #create a new Sequential Model with no layers
for layer in vgg16_model.layers:
  new_model.add(layer)   #adds all the layers of the vgg16 model to the new_model
new_model.summary()
  
  


In [9]:
new_model.layers.pop() #removes last layer (output layer) of the model

In [0]:
for layer in new_model.layers:
  layer.trainable= False     #"freezes" the layers- their weights are not updated on future training

#add new last layer which classifies into 2 categories
new_model.add(Dense(2, activation='softmax'))


In [0]:
new_model.summary()  #model now has output into 2 categories

**Training Fine-Tuned VGG16 Model**

In [0]:
new_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
#Adam is the optimization function used to optimize the loss function 


In [0]:
#model.fit(x_train_data, y_train_data, epochs, verbose) - train the model 
#Predict on data using model.predict(x_test_data)